### PyPortfolioOpt

- 종목 : KODEX 200, KODEX 단기채권, KODEX 레버리지, KODEX WTI원유선물(H), KODEX 골드선물(H), KODEX 미국S&P500선물(H)
- 상기 ETF에 대해서 PyPortfolioOpt를 사용하여 평균-분산 최적화 결과를 확인하세요
    - Sharpe Ratio 최적화

In [ ]:
import json
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

stock2code = json.load(open('../data/stock2code.json','r'))
universe = ['KODEX 200','KODEX 단기채권','KODEX 레버리지','KODEX WTI원유선물(H)','KODEX 골드선물(H)','KODEX 미국S&P500선물(H)']

df = pd.DataFrame()
for stock in universe:
    df = pd.concat([df, fdr.DataReader(stock2code[stock]).Close], axis=1)
    
df.columns = universe
df.dropna(inplace=True)
df.index = pd.to_datetime(df.index)

#<TODO>
mu = expected_returns.  # 이하의 메서드 완성하여 기대수익률 구하기
s = risk_models.        # 이하의 메서드 완성하여 공분산 구하기

ef = EfficientFrontier(mu, s)
weigths =      # ef객체의 메서드 활용하여 sharpe ratio 최대화하기
weights = ef.clean_weights()
weights

### 다팩터모델

- KODEX200에 대한 파마-프렌치 5 factor model 회귀분석
    - 아래의 csv파일은 "http://henryquant.blogspot.com/search/label/Factor%20Investing" 에서 다운로드하였음을 밝힙니다.

In [ ]:
import FinanceDataReader as fdr
import statsmodels.api as sm

# 한국시장을 기준으로 한 5 factor data를 로드한 후 KODEX200의 데이터를 total_df로 합치는 과정
three_factors = pd.read_csv('../data/five_factor_daily.csv', index_col=0, parse_dates=True).loc['2001':'2018-12-31']
kospi200 = fdr.DataReader('069500', start='2001',end='2018-12-31').Close
kospi200_ret = (np.log(kospi200) - np.log(kospi200.shift(1))) * 100
kospi200_ret.name = 'portfolio'
total_df = pd.concat([kospi200_ret, three_factors], axis=1).dropna()
total_df['Port-Rf'] = total_df['portfolio'] - total_df['Rf']


# 회귀분석을 위한 데이터 스플릿
y = total_df['Port-Rf']
X = total_df[['Mkt-Rf','SMB','HML']]
X = sm.add_constant(X)



#<TODO> : 아래의 빈칸(메서드)을 채우세요

model = sm.                # OLS 클래스를 사용하여 회귀분석 객체 생성
result = model.            # 모형 추정 메서드 사용하여 결과 객체 생성
result.                    # 결과 리포트용 메서드를 사용하여 결과 확인

### KNN

- KODEX 200에 대한 KNN모델 학습
    - 데이터분할 메서드 빈칸 채우기
    - 모델 학습 및 예측값 구하는 메서드 비칸 채우기

In [ ]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


X_data = fdr.DataReader('069500').dropna()
Y = np.where(X_data.Close.shift(-1) > X_data.Close, 1, -1)

# <TODO>
# 위에서 임포트한 sklearn 함수를 사용하여 데이터를 분할하세요
# trian : test = 0.7 : 0.3
X_train, X_test, y_train, y_test =  

tr_accuracy = []
te_accuracy = []

for k in range(2, 20):
    model = KNeighborsClassifier(n_neighbors=k)
    
    # <TODO> 
    model.              # 적당한 메서드를 사용하여 데이터 학습
    pred =              # 적당한 메서드를 사용하여 예측값을 pred변수에 저장하세요
    
    tr_accuracy.append(model.score(X_train, y_train))
    te_accuracy.append((pred==y_test).mean())
    
plt.figure(figsize=(14,5))
plt.plot(range(2,20), tr_accuracy, label='train')
plt.plot(range(2,20), te_accuracy, label='test')
plt.legend()
plt.show()